IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M4\FAULT_M4_5


In [3]:
guasto=1

VOLO m4 FAULT 5%

In [4]:
rcou_m4_fault5 = pd.read_csv("RCOU.csv")
rcou_m4_fault5 = rcou_m4_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m4_fault5 = rcou_m4_fault5[((rcou_m4_fault5['C9'] >= potenza) & (rcou_m4_fault5['C10'] >= potenza) & (rcou_m4_fault5['C11'] >= potenza) & (rcou_m4_fault5['C12'] >= potenza) & (rcou_m4_fault5['C13'] >= potenza) & (rcou_m4_fault5['C14']>= potenza))]
rcou_m4_fault5=rcou_m4_fault5.reset_index(drop=True)
display(rcou_m4_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
0,121397337,1422,1351,1373,1402,1407,1368
1,121410324,1439,1355,1377,1419,1418,1379
2,121424667,1408,1411,1412,1407,1409,1410
3,121445598,1499,1352,1382,1476,1455,1406
4,121456481,1474,1408,1412,1471,1444,1440
...,...,...,...,...,...,...,...
4788,178847914,1552,1434,1532,1458,1369,1601
4789,178858913,1555,1427,1555,1428,1410,1568
4790,178869630,1521,1463,1552,1427,1369,1596
4791,178879924,1514,1468,1544,1435,1355,1604


In [5]:
min=rcou_m4_fault5['TimeUS'][0]
max=rcou_m4_fault5['TimeUS'][len(rcou_m4_fault5)-1]
print(max)
print(min)

178933828
121397337


In [6]:
xkf1_m4_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m4_fault5 = xkf1_m4_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m4_fault5 = xkf1_m4_fault5[((xkf1_m4_fault5['TimeUS'] >= min) & (xkf1_m4_fault5['TimeUS'] <= max))]
xkf1_m4_fault5 = xkf1_m4_fault5.reset_index(drop=True)
print(xkf1_m4_fault5)

         TimeUS  Roll  Pitch    Yaw
0     121408104  1.06   0.85  11.75
1     121422491  1.01   0.87  11.74
2     121432861  1.10   0.86  11.75
3     121442909  1.16   0.87  11.74
4     121453711  1.13   0.88  11.75
...         ...   ...    ...    ...
5181  178889699 -0.12  -0.02   8.32
5182  178901435 -0.23  -0.05   8.15
5183  178911672 -0.31   0.00   8.00
5184  178921855 -0.40   0.11   7.82
5185  178931763 -0.43   0.34   7.65

[5186 rows x 4 columns]


In [7]:
att_m4_fault5 = pd.read_csv("ATT.csv")
att_m4_fault5 = att_m4_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m4_fault5 = att_m4_fault5[((att_m4_fault5['TimeUS'] >= min) & (att_m4_fault5['TimeUS'] <= max	))]
att_m4_fault5=att_m4_fault5.reset_index(drop=True)
print(att_m4_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     121409547     0.29  1.06     -0.18   0.85   11.69  11.75
1     121423615     0.29  1.01     -0.19   0.87   11.69  11.74
2     121434103     0.29  1.10     -0.20   0.86   11.69  11.75
3     121444886     0.29  1.16     -0.20   0.87   11.69  11.74
4     121455563     0.29  1.13     -0.21   0.88   11.69  11.75
...         ...      ...   ...       ...    ...     ...    ...
5181  178891074     2.47 -0.12     -0.14  -0.02    6.91   8.32
5182  178902591     2.50 -0.23     -0.22  -0.05    6.91   8.15
5183  178912882     2.54 -0.31     -0.32   0.00    6.90   8.00
5184  178922946     2.58 -0.40     -0.42   0.11    6.90   7.82
5185  178932994     2.62 -0.43     -0.51   0.34    6.90   7.65

[5186 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m4_fault5)>len(xkf1_m4_fault5):
    to_add=att_m4_fault5[len(xkf1_m4_fault5):]
    att_m4_fault5=att_m4_fault5[:len(xkf1_m4_fault5)]

if len(xkf1_m4_fault5)>len(att_m4_fault5):
    to_add=xkf1_m4_fault5[len(att_m4_fault5):]

for idx,i in enumerate(att_m4_fault5['Roll']):
    if(xkf1_m4_fault5['Roll'][idx] != i):
        att_m4_fault5['Roll'][idx] = ((att_m4_fault5['Roll'][idx] + xkf1_m4_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m4_fault5['Pitch']):
    if(xkf1_m4_fault5['Pitch'][idx] != i):
        att_m4_fault5['Pitch'][idx] = ((att_m4_fault5['Pitch'][idx] + xkf1_m4_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m4_fault5['Yaw']):
    if(xkf1_m4_fault5['Yaw'][idx] != i):
        att_m4_fault5['Yaw'][idx] = ((att_m4_fault5['Yaw'][idx] + xkf1_m4_fault5['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m4_fault5=pd.concat([att_m4_fault5,to_add])

print(att_m4_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     121409547     0.29  1.06     -0.18   0.85   11.69  11.75
1     121423615     0.29  1.01     -0.19   0.87   11.69  11.74
2     121434103     0.29  1.10     -0.20   0.86   11.69  11.75
3     121444886     0.29  1.16     -0.20   0.87   11.69  11.74
4     121455563     0.29  1.13     -0.21   0.88   11.69  11.75
...         ...      ...   ...       ...    ...     ...    ...
5181  178891074     2.47 -0.12     -0.14  -0.02    6.91   8.32
5182  178902591     2.50 -0.23     -0.22  -0.05    6.91   8.15
5183  178912882     2.54 -0.31     -0.32   0.00    6.90   8.00
5184  178922946     2.58 -0.40     -0.42   0.11    6.90   7.82
5185  178932994     2.62 -0.43     -0.51   0.34    6.90   7.65

[5186 rows x 7 columns]


C:\Users\chiar\AppData\Local\Temp\ipykernel_16872\751604635.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m4_fault5['Yaw'][idx] = ((att_m4_fault5['Yaw'][idx] + xkf1_m4_fault5['Yaw'][idx])/2)


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m4_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m4_fault5 = esc_0_m4_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m4_fault5 = esc_0_m4_fault5[((esc_0_m4_fault5['TimeUS'] >= min) & (esc_0_m4_fault5['TimeUS'] <= max))]
esc_0_m4_fault5=esc_0_m4_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m4_fault5 = pd.read_csv("ESC_1.csv")
esc_1_m4_fault5 = esc_1_m4_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m4_fault5 = esc_1_m4_fault5[((esc_1_m4_fault5['TimeUS'] >= min) & (esc_1_m4_fault5['TimeUS'] <= max))]
esc_1_m4_fault5=esc_1_m4_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m4_fault5 = pd.read_csv("ESC_2.csv")
esc_2_m4_fault5 = esc_2_m4_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m4_fault5 = esc_2_m4_fault5[((esc_2_m4_fault5['TimeUS'] >= min) & (esc_2_m4_fault5['TimeUS'] <= max))]
esc_2_m4_fault5=esc_2_m4_fault5.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m4_fault5 = pd.read_csv("ESC_3.csv")
esc_3_m4_fault5 = esc_3_m4_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m4_fault5 = esc_3_m4_fault5[((esc_3_m4_fault5['TimeUS'] >= min) & (esc_3_m4_fault5['TimeUS'] <= max))]
esc_3_m4_fault5=esc_3_m4_fault5.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m4_fault5 = pd.read_csv("ESC_4.csv")
esc_4_m4_fault5 = esc_4_m4_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m4_fault5 = esc_4_m4_fault5[((esc_4_m4_fault5['TimeUS'] >= min) & (esc_4_m4_fault5['TimeUS'] <= max))]
esc_4_m4_fault5=esc_4_m4_fault5.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m4_fault5 = pd.read_csv("ESC_5.csv")
esc_5_m4_fault5 = esc_5_m4_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m4_fault5 = esc_5_m4_fault5[((esc_5_m4_fault5['TimeUS'] >= min) & (esc_5_m4_fault5['TimeUS'] <= max))]
esc_5_m4_fault5=esc_5_m4_fault5.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m4_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m4_fault5 = imu_0_m4_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m4_fault5 = imu_0_m4_fault5[((imu_0_m4_fault5['TimeUS'] >= min) & (imu_0_m4_fault5['TimeUS'] <= max))]

imu_1_m4_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m4_fault5 = imu_1_m4_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m4_fault5 = imu_1_m4_fault5[((imu_1_m4_fault5['TimeUS'] >= min) & (imu_1_m4_fault5['TimeUS'] <= max))]

imu_2_m4_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m4_fault5 = imu_2_m4_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m4_fault5 = imu_2_m4_fault5[((imu_2_m4_fault5['TimeUS'] >= min) & (imu_2_m4_fault5['TimeUS'] <= max))]

imu_m4_fault5 = pd.concat((imu_0_m4_fault5, imu_1_m4_fault5, imu_2_m4_fault5))
imu_m4_fault5=imu_m4_fault5.groupby(imu_m4_fault5.TimeUS, as_index=False).mean()

display(imu_m4_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,121398893,0.132808,0.013634,-0.021743,-0.066052,0.333366,-10.232130
1,121401322,0.134899,0.042302,-0.007560,-0.054088,0.239980,-10.417303
2,121404530,0.053924,0.051608,0.004378,-0.022613,0.177931,-10.366177
3,121407516,0.035198,0.042260,0.006708,-0.012213,0.160736,-10.143430
4,121409376,0.102567,0.003226,-0.004532,0.010623,0.120311,-10.061591
...,...,...,...,...,...,...,...
19700,178922699,-0.208411,0.186337,-0.306661,0.568383,-0.142049,-10.251533
19701,178925360,-0.282195,0.364126,-0.281562,0.484639,0.045756,-10.359793
19702,178928239,-0.147285,0.446044,-0.268782,0.720112,0.264529,-10.229947
19703,178930888,-0.017998,0.382755,-0.279493,0.922468,0.259971,-10.184590


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m4_fault5 = pd.merge_ordered(imu_m4_fault5,att_m4_fault5)
m4_fault5=m4_fault5.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m4_fault5 = pd.merge_ordered(m4_fault5, esc_0_m4_fault5)
m4_fault5 = pd.merge_ordered(m4_fault5, esc_1_m4_fault5)
m4_fault5 = pd.merge_ordered(m4_fault5, esc_2_m4_fault5)
m4_fault5 = pd.merge_ordered(m4_fault5, esc_3_m4_fault5)
m4_fault5 = pd.merge_ordered(m4_fault5, esc_4_m4_fault5)
m4_fault5 = pd.merge_ordered(m4_fault5, esc_5_m4_fault5)
m4_fault5=m4_fault5.fillna(method="ffill").fillna(method="bfill")
m4_fault5 = pd.merge_ordered(m4_fault5, rcou_m4_fault5)
m4_fault5=m4_fault5.fillna(method="ffill").fillna(method="bfill")

In [12]:
index_to_remove=[]
for ind in att_m4_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m4_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m4_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m4_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m4_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m4_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m4_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m4_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

5186
5681
6170
6666
7143
7627
8105


In [13]:
m4_fault5 = m4_fault5[~m4_fault5.TimeUS.isin(index_to_remove)]
m4_fault5["TimeUS"] = m4_fault5["TimeUS"] - m4_fault5.iloc[0]["TimeUS"]
m4_fault5["TimeUS"] = m4_fault5["TimeUS"].astype(int)
m4_fault5["Guasto"] = guasto

In [14]:
from datetime import timedelta
m4_fault5=m4_fault5.reset_index(drop=True)
m4_fault5['TimeUS'] = pd.to_datetime(m4_fault5['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m4_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.132808,0.013634,-0.021743,-0.066052,0.333366,-10.232130,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1
1,00:00:00.002429,0.134899,0.042302,-0.007560,-0.054088,0.239980,-10.417303,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1
2,00:00:00.005637,0.053924,0.051608,0.004378,-0.022613,0.177931,-10.366177,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1
3,00:00:00.008623,0.035198,0.042260,0.006708,-0.012213,0.160736,-10.143430,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1
4,00:00:00.010483,0.102567,0.003226,-0.004532,0.010623,0.120311,-10.061591,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19700,00:00:57.523806,-0.208411,0.186337,-0.306661,0.568383,-0.142049,-10.251533,2.54,-0.31,-0.32,...,2.12,6050.0,3.02,1514.0,1468.0,1544.0,1435.0,1355.0,1604.0,1
19701,00:00:57.526467,-0.282195,0.364126,-0.281562,0.484639,0.045756,-10.359793,2.58,-0.40,-0.42,...,2.12,6050.0,3.02,1514.0,1468.0,1544.0,1435.0,1355.0,1604.0,1
19702,00:00:57.529346,-0.147285,0.446044,-0.268782,0.720112,0.264529,-10.229947,2.58,-0.40,-0.42,...,2.12,6050.0,3.02,1514.0,1468.0,1544.0,1435.0,1355.0,1604.0,1
19703,00:00:57.531995,-0.017998,0.382755,-0.279493,0.922468,0.259971,-10.184590,2.58,-0.40,-0.42,...,2.12,6050.0,3.02,1514.0,1468.0,1544.0,1435.0,1355.0,1604.0,1


## Analisi nel tempo e in frequenza

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m4_fault5)/350)):
        V1=m4_fault5[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_16872\2056186562.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_16872\2056186562.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.107776,0.038303,-1.475785,0.223180,67.0,84.0,4.584891,2.807140,0.012407,...,4589.707918,4.024160,0.792406,-0.464817,4.121166,110.0,108.0,18.647578,15.553580,1
1,1.0,-0.108410,0.038335,-1.479287,0.223559,61.0,77.0,4.660088,2.779226,0.013063,...,4436.781046,4.026477,0.792039,-0.460038,4.123385,107.0,105.0,19.477445,14.651223,1
2,2.0,-0.109070,0.038380,-1.482852,0.223980,66.0,82.0,4.594476,2.738958,0.013533,...,4743.950098,4.028782,0.791664,-0.455218,4.125591,110.0,108.0,20.301876,13.700668,1
3,3.0,-0.109694,0.038409,-1.486102,0.224351,59.0,77.0,4.486586,2.708024,0.013969,...,5066.073551,4.031073,0.791280,-0.450355,4.127784,119.0,117.0,21.119936,12.717477,1
4,4.0,-0.110351,0.038454,-1.489455,0.224773,68.0,84.0,4.497112,2.783300,0.014502,...,5401.743742,4.033352,0.790888,-0.445453,4.129962,94.0,92.0,21.930733,11.717422,1
5,5.0,-0.111021,0.038505,-1.492751,0.225217,63.0,79.0,4.599507,2.931585,0.014951,...,5749.251626,4.035618,0.790488,-0.440509,4.132128,107.0,105.0,22.733411,10.716151,1
6,6.0,-0.111777,0.038601,-1.495675,0.225803,66.0,84.0,4.660224,3.038771,0.015331,...,6106.579904,4.037871,0.790080,-0.435527,4.134280,110.0,109.0,23.527154,9.728849,1
7,7.0,-0.112562,0.038714,-1.498134,0.226442,62.0,79.0,4.617531,3.124806,0.015790,...,6471.401705,4.040112,0.789664,-0.430505,4.136419,107.0,106.0,24.311185,8.769934,1
8,8.0,-0.113371,0.038840,-1.500103,0.227123,68.0,84.0,4.534512,3.192192,0.016229,...,6841.085299,4.042340,0.789240,-0.425444,4.138545,108.0,107.0,25.084767,7.852762,1
9,9.0,-0.114119,0.038933,-1.502792,0.227702,68.0,87.0,4.561833,3.270891,0.016621,...,7212.705174,4.044556,0.788809,-0.420345,4.140658,107.0,106.0,25.847202,7.861870,1


In [18]:
path_file = path_file.replace(r"M4\FAULT_M4_5", "")
os.chdir(path_file)
df_merged.to_csv('m4_fault5.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
